In [41]:
import zipfile
import tarfile
import os

# Path to your zip file
tar_path = "DDBJ.tar.gz"

# Destination folder to extract contents
extract_to = "unzipped_DDBJ"

# Create destination folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Open and extract
#with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    #zip_ref.extractall(extract_to)
# Open and extract
with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_to)
print(f"Extracted {zip_path} to {extract_to}")


Extracted DDBJ.tar to unzipped_DDBJ


In [42]:
unzipped_DDBJ = r"C:\Users\pranjal\unzipped_DDBJ\DDBJ"

In [43]:
# Find all fasta files
snippy_dirs = [d for d in os.listdir(unzipped_DDBJ) if d.endswith(".fasta") and os.path.isfile(os.path.join(unzipped_DDBJ, d))]

In [44]:
# Collect all fasta files in that folder
fasta_files = [d for d in os.listdir(unzipped_DDBJ)
    if d.endswith(".fasta") and os.path.isfile(os.path.join(unzipped_DDBJ, d))]

In [45]:
import os
print(os.path.exists(unzipped_DDBJ))


True


In [46]:
print(fasta_files)

['AB002632.1.fasta', 'AB008550.1.fasta', 'AB009866.2.fasta', 'AB012573.1.fasta', 'AB012574.1.fasta', 'AB016282.1.fasta', 'AB043678.1.fasta', 'AB043679.1.fasta', 'AB044554.1.fasta', 'AB045978.2.fasta', 'AB063393.2.fasta', 'AB102868.1.fasta', 'AB231700.1.fasta', 'AB243556.1.fasta', 'AB255436.1.fasta', 'AB259123.2.fasta', 'AB259124.1.fasta', 'AB276040.1.fasta', 'AB285204.1.fasta', 'AB362338.2.fasta', 'AB366653.2.fasta', 'AB370205.1.fasta', 'AB370268.1.fasta', 'AB374228.1.fasta', 'AB426868.1.fasta', 'AB434711.1.fasta', 'AB451219.1.fasta', 'AB472900.2.fasta', 'AB560486.1.fasta', 'AB572858.1.fasta', 'AB597179.1.fasta', 'AB609718.1.fasta', 'AB626962.1.fasta', 'AB626963.1.fasta', 'AB647160.1.fasta', 'AB711120.1.fasta', 'AB712291.1.fasta', 'AB716666.1.fasta', 'AB720063.2.fasta', 'AB720064.1.fasta', 'AB731695.1.fasta', 'AB746912.1.fasta', 'AB757800.1.fasta', 'AB757801.1.fasta', 'AB767244.1.fasta', 'AB797215.1.fasta', 'AB823818.1.fasta', 'AB828698.1.fasta', 'AB854109.1.fasta', 'AB863625.1.fasta',

In [47]:
from Bio import SeqIO
import pandas as pd

In [48]:
# Loop through each file and parse
for fasta in fasta_files:
    fasta_path = os.path.join(unzipped_DDBJ, fasta)
    for record in SeqIO.parse(fasta_path, "fasta"):
        print(record.id, len(record.seq))

AB002632.1 8651
AB008550.1 35580
AB009866.2 41401
AB012573.1 7965
AB012574.1 7965
AB016282.1 39325
AB043678.1 8784
AB043679.1 6891
AB044554.1 45636
AB045978.2 42942
AB063393.2 19604
AB102868.1 41391
AB231700.1 162109
AB243556.1 44857
AB255436.1 60238
AB259123.2 9004
AB259124.1 6662
AB276040.1 38760
AB285204.1 47021
AB362338.2 41660
AB366653.2 231255
AB370205.1 44342
AB370268.1 43011
AB374228.1 33507
AB426868.1 41660
AB434711.1 8929
AB451219.1 43079
AB472900.2 88322
AB560486.1 64144
AB572858.1 6856
AB597179.1 40411
AB609718.1 142072
AB626962.1 18168
AB626963.1 18186
AB647160.1 26215
AB711120.1 50861
AB712291.1 53996
AB716666.1 43871
AB720063.2 43870
AB720064.1 42963
AB731695.1 26449
AB746912.1 37159
AB757800.1 41549
AB757801.1 41684
AB767244.1 42746
AB797215.1 159991
AB823818.1 38099
AB828698.1 8576
AB854109.1 44578
AB863625.1 40471
AB903967.1 142094
AB910602.1 7325
AB920995.1 44360
AB931172.1 6386
AB937974.1 7679
AB971354.1 4217
AB981169.1 40002
AB981170.1 8956
AP000363.1 60942
AP00155

In [49]:
def infer_sequence_type(header, length):
    h = header.lower()
    if "chromosome" in h or "chr" in h:
        return "Chromosome"
    if "plasmid" in h:
        return "Plasmid"
    if "scaffold" in h:
        return "Scaffold"
    if "contig" in h:
        return "Contig"
    
    # length-based fallback logic
    if length > 1_000_000:
        return "Chromosome"
    elif length > 50_000:
        return "Contig"
    else:
        return "Small_Contig"

def infer_topology(header):
    h = header.lower()
    if "circular" in h:
        return "Circular"
    if "linear" in h:
        return "Linear"
    return "Unknown"

def infer_plasmid_or_chromosome(header):
    h = header.lower()
    if "plasmid" in h:
        return "Plasmid"
    if "chromosome" in h or "chr" in h:
        return "Chromosome"
    return "Unknown"

In [50]:
seq_ids = []
seq_lengths = []
file_names = []
gc_contents = []
at_contents = []
n_counts = []
gc_skews = []
at_skews = []
sequence_types = []
topologies = []
plasmid_chromosome = []
for fasta in fasta_files:
    fasta_path = os.path.join(unzipped_DDBJ, fasta)
    for record in SeqIO.parse(fasta_path, "fasta"):
        seq = str(record.seq).upper()
        seq_ids.append(record.id)
        seq_lengths.append(len(record.seq))
        gc = seq.count("G") + seq.count("C")
        at = seq.count("A") + seq.count("T")
        gc_contents.append((seq.count("G") + seq.count("C")) / len(seq) * 100 if len(seq) > 0 else 0)
        at_contents.append((seq.count("A") + seq.count("T")) / len(seq) * 100 if len(seq) > 0 else 0)
        gc_skews.append((seq.count("G") - seq.count("C")) / (gc) * 100 if gc > 0 else 0)
        at_skews.append((seq.count("A") - seq.count("T")) / (at) * 100 if at > 0 else 0)
        header = record.description
        sequence_types.append(infer_sequence_type(header, len(seq)))
        topologies.append(infer_topology(header))
        plasmid_chromosome.append(infer_plasmid_or_chromosome(header))
        n_counts.append(seq.count("N"))
        file_names.append(fasta)

In [51]:
# Create DataFrame
df = pd.DataFrame({
    "File": file_names,
    "Sequence ID": seq_ids,
    "Sequence Length": seq_lengths,
    "GC_CONTENT" : gc_contents,
    "AT_CONTENT" : at_contents,
    "Ambigious_Base" : n_counts,
    "GC_SKEWS" : gc_skews ,
    "AT_SKEWS" : at_skews ,
    "Sequence_type" : sequence_types,
    "Topology" : topologies, 
    "Chromosome/Plasmid" : plasmid_chromosome
    
})

In [53]:
df.to_csv("sequence_summary1.csv", index=False)
print(df.head())

               File Sequence ID  Sequence Length  GC_CONTENT  AT_CONTENT  \
0  AB002632.1.fasta  AB002632.1             8651   44.526644   55.473356   
1  AB008550.1.fasta  AB008550.1            35580   62.622260   37.377740   
2  AB009866.2.fasta  AB009866.2            41401   33.554745   66.445255   
3  AB012573.1.fasta  AB012573.1             7965   45.662272   54.337728   
4  AB012574.1.fasta  AB012574.1             7965   45.674827   54.325173   

   Ambigious_Base   GC_SKEWS   AT_SKEWS Sequence_type Topology  \
0               0  -5.140187  -5.021880  Small_Contig  Unknown   
1               0  -6.871325  10.023310  Small_Contig  Unknown   
2               0  14.655917  12.574067  Small_Contig  Unknown   
3               0  -1.732197  -6.284658  Small_Contig  Unknown   
4               0  -1.759208  -6.263000  Small_Contig  Unknown   

  Chromosome/Plasmid  
0            Unknown  
1            Unknown  
2            Unknown  
3            Unknown  
4            Unknown  


## Extracting Sequence information from genbank files with multiple sequence:

## Getting features information

In [5]:
from Bio import SeqIO

# Parse multiple genome records from a GenBank file
for genome_record in SeqIO.parse("sequences_test.gb", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    print("Features:")
    for feature in genome_record.features:
        print(f" - Type: {feature.type}")
        print(f"   Location: {feature.location}")
        print(f"   Qualifiers: {feature.qualifiers}")
    print("-" * 60)


Genome ID: MH746957.1
Features:
 - Type: source
   Location: [0:1032](+)
   Qualifiers: {'organism': ['Asian prunus virus 3'], 'mol_type': ['genomic RNA'], 'host': ['apricot'], 'db_xref': ['taxon:351428'], 'country': ['China'], 'collection_date': ['10-May-2018']}
 - Type: misc_feature
   Location: [64:>1032](+)
   Qualifiers: {'note': ['similar to coat protein']}
------------------------------------------------------------
Genome ID: MH746955.1
Features:
 - Type: source
   Location: [0:1031](+)
   Qualifiers: {'organism': ['Asian prunus virus 3'], 'mol_type': ['genomic RNA'], 'host': ['apricot'], 'db_xref': ['taxon:351428'], 'country': ['China'], 'collection_date': ['10-May-2018']}
 - Type: misc_feature
   Location: [64:>1031](+)
   Qualifiers: {'note': ['similar to coat protein']}
------------------------------------------------------------
Genome ID: MH476301.1
Features:
 - Type: source
   Location: [0:3207](+)
   Qualifiers: {'organism': ['Grapevine red blotch virus'], 'mol_type': [

## Parsing Genbank features metadata

In [22]:
from Bio import SeqIO
import pandas as pd

# List to collect feature-level data
feature_rows = []

# Parse GenBank file
for genome_record in SeqIO.parse("sequences_test.gb", "genbank"):
    genome_id = genome_record.id

    for feature in genome_record.features:
        row = {
            "GenomeID": genome_id,
            "FeatureType": feature.type,
            "Location": str(feature.location)
        }

        # Add common qualifiers if present
        for key in ["locus_tag", "gene", "product", "protein_id", "isolate", "host", "country", "segment", "db_xref"]:
            row[key] = ";".join(feature.qualifiers.get(key, ["nd"]))

        feature_rows.append(row)

# Convert to DataFrame
df_features = pd.DataFrame(feature_rows)

# Preview
print(df_features.head())

# Save to CSV
df_features.to_csv("genbank_features_table.csv", index=False)


     GenomeID   FeatureType       Location locus_tag gene product protein_id  \
0  MH746957.1        source    [0:1032](+)        nd   nd      nd         nd   
1  MH746957.1  misc_feature  [64:>1032](+)        nd   nd      nd         nd   
2  MH746955.1        source    [0:1031](+)        nd   nd      nd         nd   
3  MH746955.1  misc_feature  [64:>1031](+)        nd   nd      nd         nd   
4  MH476301.1        source    [0:3207](+)        nd   nd      nd         nd   

  isolate                           host country segment        db_xref  
0      nd                        apricot   China      nd   taxon:351428  
1      nd                             nd      nd      nd             nd  
2      nd                        apricot   China      nd   taxon:351428  
3      nd                             nd      nd      nd             nd  
4   NSRB8  Vitis vinifera cv. Pinot Noir  Canada      nd  taxon:1381007  


## Parse multiple genome records from a GenBank file

In [7]:
from Bio import SeqIO

# Parse multiple genome records from a GenBank file
for record in SeqIO.parse("sequences_test.gb", "genbank"):
    print("=" * 60)
    print(f"Genome ID: {record.id}")
    print(f"Description: {record.description}")
    print(f"Organism: {record.annotations.get('organism', 'nd')}")
    print(f"Keywords: {record.annotations.get('keywords', [])}")
    print(f"Sequence length: {len(record.seq)} bp")

    # GC content
    seq = record.seq
    gc_content = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)
    print(f"GC Content: {round(gc_content, 2)}%")

    # Extract from features
    for feature in record.features:
        if feature.type == "source":
            print(f"Isolate: {feature.qualifiers.get('isolate', ['nd'])[0]}")
            print(f"Host: {feature.qualifiers.get('host', ['nd'])[0]}")
            print(f"Country: {feature.qualifiers.get('country', ['nd'])[0]}")
            print(f"Segment: {feature.qualifiers.get('segment', ['nd'])[0]}")

        if feature.type == "CDS":
            print(f"Family: {feature.qualifiers.get('family', ['nd'])[0]}")
            print(f"Genus: {feature.qualifiers.get('genus', ['nd'])[0]}")
            print(f"PubMed/DBxref: {feature.qualifiers.get('db_xref', ['nd'])[0]}")

    print("=" * 60)


Genome ID: MH746957.1
Description: UNVERIFIED: Asian prunus virus 3 coat protein-like gene, partial sequence
Organism: Asian prunus virus 3
Keywords: ['UNVERIFIED']
Sequence length: 1032 bp
GC Content: 48.45%
Isolate: nd
Host: apricot
Country: China
Segment: nd
Genome ID: MH746955.1
Description: UNVERIFIED: Asian prunus virus 3 coat protein-like gene, partial sequence
Organism: Asian prunus virus 3
Keywords: ['UNVERIFIED']
Sequence length: 1031 bp
GC Content: 48.88%
Isolate: nd
Host: apricot
Country: China
Segment: nd
Genome ID: MH476301.1
Description: UNVERIFIED: Grapevine red blotch virus isolate NSRB8, complete genome
Organism: Grapevine red blotch virus
Keywords: ['UNVERIFIED']
Sequence length: 3207 bp
GC Content: 42.22%
Isolate: NSRB8
Host: Vitis vinifera cv. Pinot Noir
Country: Canada
Segment: nd
Family: nd
Genus: nd
PubMed/DBxref: nd
Family: nd
Genus: nd
PubMed/DBxref: nd
Family: nd
Genus: nd
PubMed/DBxref: nd
Family: nd
Genus: nd
PubMed/DBxref: nd
Genome ID: MH476299.1
Descript

## Finding Locus tags

In [11]:
from Bio import SeqIO

for record in SeqIO.parse("sequences_test.gb", "genbank"):
    print(f"Genome ID: {record.id}")
    locus_tags = []
    for feature in record.features:
        # locus_tag is usually in CDS or gene features
        if "locus_tag" in feature.qualifiers:
            locus_tags.append(feature.qualifiers["locus_tag"][0])
    print("Locus tags:", locus_tags)
    print("-" * 60)


Genome ID: MH746957.1
Locus tags: []
------------------------------------------------------------
Genome ID: MH746955.1
Locus tags: []
------------------------------------------------------------
Genome ID: MH476301.1
Locus tags: []
------------------------------------------------------------
Genome ID: MH476299.1
Locus tags: []
------------------------------------------------------------
Genome ID: NC_040817.1
Locus tags: ['EXL43_gp1', 'EXL43_gp1', 'EXL43_gp2', 'EXL43_gp2', 'EXL43_gp3', 'EXL43_gp3', 'EXL43_gp4', 'EXL43_gp4', 'EXL43_gp5', 'EXL43_gp5', 'EXL43_gp6', 'EXL43_gp6']
------------------------------------------------------------
Genome ID: NC_040757.1
Locus tags: ['F2Q04_s1gp1', 'F2Q04_s1gp1']
------------------------------------------------------------
Genome ID: NC_040756.1
Locus tags: ['F2Q04_s2gp1', 'F2Q04_s2gp1']
------------------------------------------------------------
Genome ID: NC_040751.1
Locus tags: ['F2P95_sCgp1', 'F2P95_sCgp1']
-----------------------------------

## Parse GenBank file

In [10]:
from Bio import SeqIO

# Initialize lists
length = []
code = []
sequence_l = []
description = []
isolate_l = []
host_l = []
country_l = []
segment_l = []
gc = []
organism_l = []
keyword_l = []
family_l = []
genus_l = []
pubmed_l = []

# Parse GenBank file
for record in SeqIO.parse("sequences_test.gb", "genbank"):
    # Basic attributes
    length.append(len(record.seq))
    code.append(record.id)
    description.append(record.description)
    organism_l.append(record.annotations.get("organism", "nd"))
    keyword_l.append(record.annotations.get("keywords", []))

    # Sequence
    sequence_l.append(str(record.seq))

    # GC content
    seq = record.seq
    gc_content = 100.0 * float(seq.count("G") + seq.count("C")) / len(seq)
    gc.append(round(gc_content, 2))

    # Features
    for feature in record.features:
        if feature.type == "source":
            isolate_l.append(feature.qualifiers.get("isolate", ["nd"])[0])
            host_l.append(feature.qualifiers.get("host", ["nd"])[0])
            country_l.append(feature.qualifiers.get("country", ["nd"])[0])
            segment_l.append(feature.qualifiers.get("segment", ["nd"])[0])
        if feature.type == "CDS":
            family_l.append(feature.qualifiers.get("family", ["nd"])[0])
            genus_l.append(feature.qualifiers.get("genus", ["nd"])[0])
            pubmed_l.append(feature.qualifiers.get("db_xref", ["nd"])[0])

# Example: print summary
#Display only top 5
#print("Codes:", code[:5])
print("GenomeIDs:", code)
print("Organisms:", organism_l)
print("Hosts:", host_l)
print("GC Content:", gc)


GenomeIDs: ['MH746957.1', 'MH746955.1', 'MH476301.1', 'MH476299.1', 'NC_040817.1', 'NC_040757.1', 'NC_040756.1', 'NC_040751.1', 'NC_040750.1', 'NC_040749.1', 'NC_040748.1', 'NC_040747.1', 'NC_040746.1', 'NC_040745.1']
Organisms: ['Asian prunus virus 3', 'Asian prunus virus 3', 'Grapevine red blotch virus', 'Grapevine red blotch virus', 'Camellia chlorotic dwarf-associated virus', 'Aspergillus fumigatus partitivirus 2', 'Aspergillus fumigatus partitivirus 2', 'Cow vetch latent virus', 'Cow vetch latent virus', 'Cow vetch latent virus', 'Cow vetch latent virus', 'Cow vetch latent virus', 'Cow vetch latent virus', 'Cow vetch latent virus']
Hosts: ['apricot', 'apricot', 'Vitis vinifera cv. Pinot Noir', 'Vitis vinifera cv. New York Muscat', 'camellia', 'Aspergillus fumigatus', 'Aspergillus fumigatus', 'Vicia cracca', 'Vicia cracca', 'Vicia cracca', 'Vicia cracca', 'Vicia cracca', 'Vicia cracca', 'Vicia cracca']
GC Content: [48.45, 48.88, 42.22, 42.24, 41.17, 46.38, 47.99, 38.11, 35.92, 36.2

## converting to dataframe and saving as csv

In [20]:
from Bio import SeqIO
import pandas as pd

# Prepare a list of dictionaries (rows for DataFrame)
records_data = []

for record in SeqIO.parse("sequences_test.gb", "genbank"):
    # Basic attributes
    genome_id = record.id
    description = record.description
    organism = record.annotations.get("organism", "nd")
    keywords = ";".join(record.annotations.get("keywords", []))  # join list into string
    seq_length = len(record.seq)

    # GC content
    seq = record.seq
    gc_content = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)

    # Defaults
    isolate = host = country = segment = "nd"
    family = genus = pubmed = "nd"

    # Extract from features
    for feature in record.features:
        if feature.type == "source":
            isolate = feature.qualifiers.get("isolate", ["nd"])[0]
            host = feature.qualifiers.get("host", ["nd"])[0]
            country = feature.qualifiers.get("country", ["nd"])[0]
            segment = feature.qualifiers.get("segment", ["nd"])[0]

        if feature.type == "CDS":
            family = feature.qualifiers.get("family", ["nd"])[0]
            genus = feature.qualifiers.get("genus", ["nd"])[0]
            pubmed = feature.qualifiers.get("db_xref", ["nd"])[0]

    # Append row to list
    records_data.append({
        "GenomeID": genome_id,
        "Description": description,
        "Organism": organism,
        "Keywords": keywords,
        "SequenceLength": seq_length,
        "GC_Content": round(gc_content, 2),
        "Isolate": isolate,
        "Host": host,
        "Country": country,
        "Segment": segment,
        "Family": family,
        "Genus": genus,
        "PubMed_DBxref": pubmed
    })

# Convert to DataFrame
df = pd.DataFrame(records_data)

# Save to CSV
df.to_csv("genbank_metadata.csv", index=False)

# Preview
print(df.head())


      GenomeID                                        Description  \
0   MH746957.1  UNVERIFIED: Asian prunus virus 3 coat protein-...   
1   MH746955.1  UNVERIFIED: Asian prunus virus 3 coat protein-...   
2   MH476301.1  UNVERIFIED: Grapevine red blotch virus isolate...   
3   MH476299.1  UNVERIFIED: Grapevine red blotch virus isolate...   
4  NC_040817.1  Camellia chlorotic dwarf-associated virus isol...   

                                    Organism    Keywords  SequenceLength  \
0                       Asian prunus virus 3  UNVERIFIED            1032   
1                       Asian prunus virus 3  UNVERIFIED            1031   
2                 Grapevine red blotch virus  UNVERIFIED            3207   
3                 Grapevine red blotch virus  UNVERIFIED            3208   
4  Camellia chlorotic dwarf-associated virus      RefSeq            3687   

   GC_Content Isolate                                Host Country Segment  \
0       48.45      nd                             a

## Function to look for CDS feature by annotation value

In [13]:
# Biopython's SeqIO module handles sequence input/output
from Bio import SeqIO

def get_cds_feature_with_qualifier_value(seq_record, name, value):
    """Function to look for CDS feature by annotation value in sequence record.
    
    e.g. You can use this for finding features by locus tag, gene ID, or protein ID.
    """
    # Loop over the features
    for feature in genome_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    # Could not find it
    return None

# Parse multiple genome records from a GenBank file
for genome_record in SeqIO.parse("sequences_test.gb", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "EXL43_gp1")
    if cds_feature:
        print("Found CDS feature:", cds_feature)
    else:
        print("No matching CDS feature found.")
    print("-" * 60)

Genome ID: MH746957.1
No matching CDS feature found.
------------------------------------------------------------
Genome ID: MH746955.1
No matching CDS feature found.
------------------------------------------------------------
Genome ID: MH476301.1
No matching CDS feature found.
------------------------------------------------------------
Genome ID: MH476299.1
No matching CDS feature found.
------------------------------------------------------------
Genome ID: NC_040817.1
Found CDS feature: type: CDS
location: [234:561](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:41701981']
    Key: locus_tag, Value: ['EXL43_gp1']
    Key: product, Value: ['hypothetical protein']
    Key: protein_id, Value: ['YP_009553588.1']
    Key: translation, Value: ['MFRVRHVLLAISKLWVFELKYMEKIFGDTESENKVLFALCILFIIYGLVCNVVFCYGGFTRKYMGPSIYVLREVSDGGVECCSSCPSVNPGGYADGEVGISRKKLPEC']

------------------------------------------------------------
Genome ID: NC_040757.1
No matching 

## similar to above

In [16]:
for genome_record in SeqIO.parse("sequences_test.gb", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "EXL43_gp1")
    print(cds_feature)

Genome ID: MH746957.1
None
Genome ID: MH746955.1
None
Genome ID: MH476301.1
None
Genome ID: MH476299.1
None
Genome ID: NC_040817.1
type: CDS
location: [234:561](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:41701981']
    Key: locus_tag, Value: ['EXL43_gp1']
    Key: product, Value: ['hypothetical protein']
    Key: protein_id, Value: ['YP_009553588.1']
    Key: translation, Value: ['MFRVRHVLLAISKLWVFELKYMEKIFGDTESENKVLFALCILFIIYGLVCNVVFCYGGFTRKYMGPSIYVLREVSDGGVECCSSCPSVNPGGYADGEVGISRKKLPEC']

Genome ID: NC_040757.1
None
Genome ID: NC_040756.1
None
Genome ID: NC_040751.1
None
Genome ID: NC_040750.1
None
Genome ID: NC_040749.1
None
Genome ID: NC_040748.1
None
Genome ID: NC_040747.1
None
Genome ID: NC_040746.1
None
Genome ID: NC_040745.1
None


## Get the start, stop codon of cds 

In [19]:
from Bio import SeqIO

def get_cds_feature_with_qualifier_value(seq_record, name, value):
    for feature in seq_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    return None

for genome_record in SeqIO.parse("sequences_test.gb", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "EXL43_gp1")
    
    if cds_feature:  # ✅ check before using
        gene_sequence = cds_feature.extract(genome_record.seq)
        print("CDS nucleotide sequence:")
        print(gene_sequence)
        print("Start codon is %s" % gene_sequence[:3])
        print("Stop codon is %s" % gene_sequence[-3:])
    else:
        print("No CDS feature with locus_tag = EXL43_gp1 found.")
    print("-" * 60)


Genome ID: MH746957.1
No CDS feature with locus_tag = EXL43_gp1 found.
------------------------------------------------------------
Genome ID: MH746955.1
No CDS feature with locus_tag = EXL43_gp1 found.
------------------------------------------------------------
Genome ID: MH476301.1
No CDS feature with locus_tag = EXL43_gp1 found.
------------------------------------------------------------
Genome ID: MH476299.1
No CDS feature with locus_tag = EXL43_gp1 found.
------------------------------------------------------------
Genome ID: NC_040817.1
CDS nucleotide sequence:
ATGTTTCGTGTAAGACACGTTTTGTTGGCTATAAGTAAGTTGTGGGTGTTTGAACTGAAATATATGGAGAAAATATTTGGAGATACAGAGAGCGAAAATAAGGTATTATTTGCACTGTGCATTTTATTTATTATTTACGGTTTAGTTTGCAATGTCGTCTTTTGCTATGGAGGGTTTACCCGAAAGTATATGGGGCCTAGTATTTATGTGTTGCGTGAGGTATCTGATGGGGGAGTTGAATGCTGTTCTTCCTGCCCGTCAGTTAATCCCGGAGGTTATGCCGATGGGGAAGTTGGGATTTCTCGGAAAAAATTGCCAGAGTGTTAA
Start codon is ATG
Stop codon is TAA
-----------------------------------------------------------

## Repeat for prokka annotated sequence

In [38]:
from Bio import SeqIO

def get_cds_feature_with_qualifier_value(seq_record, name, value):
    for feature in seq_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    return None

for genome_record in SeqIO.parse("Acinetobacter.gbk", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "DGLPADFO_00003")
    
    if cds_feature:  # ✅ check before using
        gene_sequence = cds_feature.extract(genome_record.seq)
        print("CDS nucleotide sequence:")
        print(gene_sequence)
        print("Start codon is %s" % gene_sequence[:3])
        print("Stop codon is %s" % gene_sequence[-3:])
    else:
        print("No CDS feature with locus_tag = EXL43_gp1 found.")
    print("-" * 60)


Genome ID: NZ_CP045110.1
CDS nucleotide sequence:
ATGGATCTGACCAAACGTATTCATCAATACACACGTGGGGCTTGGACATTCGATGTCATTGATTCAGGCCCTATAGATGGCCCTCTCGTGGTTTTATTACATGGCTTTCCAGAAACTGCCCATAGTTGGGAGCAGACTTCGGAATTATTGCATGAGCATCAATTTAGAACATTTGCGATCCATCAACGTGGTTATAGTCTAAATGCTAGACCTCAAAGCCGCTTTCATTACGCTTTAAGTGAACTAACTGAAGATGTCGTAGTTTTTATTAAGGGACTTGGTCAACCAGTTTATTTAATTGGGCACGATTGGGGTTCTGTAGTTGCGTCTAGTGTAGCTATGAAGTATCCGCAGTATATTCAACATCTCACTTTAGTTTCCGTACCCCATCAAGCTGCTTTTTTAAAAGCCTGCTTAAGTAGCAATCAACTACTAAAATCTTATTATTTTGCAGTTTTTCAGCTTCCTATTTTGCCCGAACTTTTATTTAAGAAAATGCCTAAATTAGGCCGTAACTTTCTTAAATCATCTGGCATGACCGAAAAGCAAATTAAAATATTCGAGACTGAGTTTATAAAAGAAAATAGGCTAACTACGGCATTAAACTGGTATCGTGGCTTTTTTTGGGACAAACCTGAAAATCCTTTTAAAGCAATAGACGTTCCCACTCTTTTTATTTGGGGTAAGCATGATATTGCAGTAACAGAAAAAAGTGCCAAGTTAAATTCAGGTTATTTTAAAAATAATTATAAGGCTATTTTTATGGATGCCTCTCATTGGATTCCTTATCAAAATGCACCTGAACTCGTACAATGCTTTTTAGAATCTATACATGATCAAGCATAA
Start codon is ATG
Stop codon is TAA
------------------------------------------------------------
Genom

## Using Prokka generated file with single sequence

In [23]:
from Bio import SeqIO

# Parse multiple genome records from a GenBank file
for genome_record in SeqIO.parse("pseudomonas_prokka.gbk", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    print("Features:")
    for feature in genome_record.features:
        print(f" - Type: {feature.type}")
        print(f"   Location: {feature.location}")
        print(f"   Qualifiers: {feature.qualifiers}")
    print("-" * 60)


Genome ID: NZ_CP045110.1
Features:
 - Type: source
   Location: [0:3981941](+)
   Qualifiers: {'organism': ['Genus species'], 'mol_type': ['genomic DNA'], 'strain': ['strain']}
 - Type: CDS
   Location: [348:1563](-)
   Qualifiers: {'gene': ['tyrS'], 'locus_tag': ['DGLPADFO_00001'], 'EC_number': ['6.1.1.1'], 'inference': ['ab initio prediction:Prodigal:002006', 'similar to AA sequence:UniProtKB:P41256'], 'codon_start': ['1'], 'transl_table': ['11'], 'product': ['Tyrosine--tRNA ligase'], 'db_xref': ['COG:COG0162'], 'translation': ['MSNFLPAEEQLALIQRGTHEIISEEDLLKKLKENRPLKIKAGFDPTAPDLHLGHTVLINKLKTFQDLGHEVTFLIGDYTAMIGDPTGKSATRPPLSREQVEANAKTYQEQVFKILDPNKTKVRFNSEWFNQKSAADLIQLASQQTVSRMLERDDFTKRYNNHQPIAIHEFLYPLVQGYDSIALEADVELGGTDQTFNLLMGRTLQSRYGQESQVCITVPILEGLDGVNKMSKSLGNYIGVFDAPGAMYQKVLSMPDSLIERYFDLLSFKSLDEIKALLDEIAAGRNPQEVKRILALELVERFHDAEAAANAHKSAGNRITEGEVPADTPEVTISRGEFGGEIFIATILRVAGLNPNAAAAKDAVARGAVKVDWNVVDASFSVKENGTFIIQSGKKAIARVTFTD']}
 - Type: CDS
   Location: [1642:2770](+)
   Qualifiers:

## Generating Locus tags

In [24]:
from Bio import SeqIO

for record in SeqIO.parse("pseudomonas_prokka.gbk", "genbank"):
    print(f"Genome ID: {record.id}")
    locus_tags = []
    for feature in record.features:
        # locus_tag is usually in CDS or gene features
        if "locus_tag" in feature.qualifiers:
            locus_tags.append(feature.qualifiers["locus_tag"][0])
    print("Locus tags:", locus_tags)
    print("-" * 60)


Genome ID: NZ_CP045110.1
Locus tags: ['DGLPADFO_00001', 'DGLPADFO_00002', 'DGLPADFO_00003', 'DGLPADFO_00004', 'DGLPADFO_00005', 'DGLPADFO_00006', 'DGLPADFO_00007', 'DGLPADFO_00008', 'DGLPADFO_00009', 'DGLPADFO_00010', 'DGLPADFO_00011', 'DGLPADFO_00012', 'DGLPADFO_00013', 'DGLPADFO_00014', 'DGLPADFO_00015', 'DGLPADFO_00016', 'DGLPADFO_00017', 'DGLPADFO_00018', 'DGLPADFO_00019', 'DGLPADFO_00020', 'DGLPADFO_00021', 'DGLPADFO_00022', 'DGLPADFO_00023', 'DGLPADFO_00024', 'DGLPADFO_00025', 'DGLPADFO_00026', 'DGLPADFO_00027', 'DGLPADFO_00028', 'DGLPADFO_00029', 'DGLPADFO_00030', 'DGLPADFO_00031', 'DGLPADFO_00032', 'DGLPADFO_00033', 'DGLPADFO_00034', 'DGLPADFO_00035', 'DGLPADFO_00036', 'DGLPADFO_00037', 'DGLPADFO_00038', 'DGLPADFO_00039', 'DGLPADFO_00040', 'DGLPADFO_00041', 'DGLPADFO_00042', 'DGLPADFO_00043', 'DGLPADFO_00044', 'DGLPADFO_00045', 'DGLPADFO_00046', 'DGLPADFO_00047', 'DGLPADFO_00048', 'DGLPADFO_00049', 'DGLPADFO_00050', 'DGLPADFO_00051', 'DGLPADFO_00052', 'DGLPADFO_00053', 'DGLPADF

## Using Locus tag for finding cds_feature

In [25]:
# Biopython's SeqIO module handles sequence input/output
from Bio import SeqIO

def get_cds_feature_with_qualifier_value(seq_record, name, value):
    """Function to look for CDS feature by annotation value in sequence record.
    
    e.g. You can use this for finding features by locus tag, gene ID, or protein ID.
    """
    # Loop over the features
    for feature in genome_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    # Could not find it
    return None

# Parse multiple genome records from a GenBank file
for genome_record in SeqIO.parse("pseudomonas_prokka.gbk", "genbank"):
    print(f"Genome ID: {genome_record.id}")
    cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "DGLPADFO_00002")
    if cds_feature:
        print("Found CDS feature:", cds_feature)
    else:
        print("No matching CDS feature found.")
    print("-" * 60)

Genome ID: NZ_CP045110.1
Found CDS feature: type: CDS
location: [1642:2770](+)
qualifiers:
    Key: EC_number, Value: ['2.7.1.170']
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['COG:COG2377']
    Key: gene, Value: ['anmK']
    Key: inference, Value: ['ab initio prediction:Prodigal:002006', 'similar to AA sequence:UniProtKB:Q8EHB5']
    Key: locus_tag, Value: ['DGLPADFO_00002']
    Key: product, Value: ['Anhydro-N-acetylmuramic acid kinase']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MSAIYIGVMTGTSMDGVDIVAASFEPLQLHATLTVPFEPELRDELMALTLPDDNEIDRMGKADVALAKMIGHGINSLIEKHQLDRSQIKAIGSHGQTIRHRPEHGFTLQIGDPNIITEITQIPVVSDFRRRDMAAGGQGAPLVPAFHQALFQHPSIHRVILNLGGIANVSMLPANNPDGVFGFDTGPANILMDAWCHRHTGHPYDENGDWAAYGHPIRSLLDRLYAHEYFSKEPPKSTGREDFNIDWLDDQLIDWRNDLTYDELEDTPENIQATLLKLTVRAIQKAIYRSNMETGEVYVCGGGAYNSYLLEQLRWRLRKHNWSVQTTDVLGLSPTWVEATAFAWLAMRFVDGLSGNLPAVTGASDFRILGTITAV']

------------------------------------------------------------
Genome ID: NZ_CP045108.1
No mat

## Creating a dataframe for getting basic metadata

In [26]:
from Bio import SeqIO
import pandas as pd

# Prepare a list of dictionaries (rows for DataFrame)
records_data = []

for record in SeqIO.parse("pseudomonas_prokka.gbk", "genbank"):
    # Basic attributes
    genome_id = record.id
    description = record.description
    organism = record.annotations.get("organism", "nd")
    keywords = ";".join(record.annotations.get("keywords", []))  # join list into string
    seq_length = len(record.seq)

    # GC content
    seq = record.seq
    gc_content = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)

    # Defaults
    isolate = host = country = segment = "nd"
    family = genus = pubmed = "nd"

    # Extract from features
    for feature in record.features:
        if feature.type == "source":
            isolate = feature.qualifiers.get("isolate", ["nd"])[0]
            host = feature.qualifiers.get("host", ["nd"])[0]
            country = feature.qualifiers.get("country", ["nd"])[0]
            segment = feature.qualifiers.get("segment", ["nd"])[0]

        if feature.type == "CDS":
            family = feature.qualifiers.get("family", ["nd"])[0]
            genus = feature.qualifiers.get("genus", ["nd"])[0]
            pubmed = feature.qualifiers.get("db_xref", ["nd"])[0]

    # Append row to list
    records_data.append({
        "GenomeID": genome_id,
        "Description": description,
        "Organism": organism,
        "Keywords": keywords,
        "SequenceLength": seq_length,
        "GC_Content": round(gc_content, 2),
        "Isolate": isolate,
        "Host": host,
        "Country": country,
        "Segment": segment,
        "Family": family,
        "Genus": genus,
        "PubMed_DBxref": pubmed
    })

# Convert to DataFrame
df = pd.DataFrame(records_data)

# Save to CSV
df.to_csv("pseudomonas_metadata.csv", index=False)

# Preview
print(df.head())


        GenomeID                  Description       Organism Keywords  \
0  NZ_CP045110.1  Genus species strain strain  Genus species            
1  NZ_CP045108.1  Genus species strain strain  Genus species            
2  NZ_CP045109.1  Genus species strain strain  Genus species            

   SequenceLength  GC_Content Isolate Host Country Segment Family Genus  \
0         3981941       39.17      nd   nd      nd      nd     nd    nd   
1            7655       33.39      nd   nd      nd      nd     nd    nd   
2            9540       34.68      nd   nd      nd      nd     nd    nd   

  PubMed_DBxref  
0   COG:COG0625  
1            nd  
2            nd  


## Generating Feature-rich Meta data

In [27]:
from Bio import SeqIO
import pandas as pd

# List to collect feature-level data
feature_rows = []

# Parse GenBank file
for genome_record in SeqIO.parse("pseudomonas_prokka.gbk", "genbank"):
    genome_id = genome_record.id

    for feature in genome_record.features:
        row = {
            "GenomeID": genome_id,
            "FeatureType": feature.type,
            "Location": str(feature.location)
        }

        # Add common qualifiers if present
        for key in ["locus_tag", "gene", "product", "protein_id", "isolate", "host", "country", "segment", "db_xref"]:
            row[key] = ";".join(feature.qualifiers.get(key, ["nd"]))

        feature_rows.append(row)

# Convert to DataFrame
df_features = pd.DataFrame(feature_rows)

# Preview
print(df_features.head())

# Save to CSV
df_features.to_csv("pseudomonas_features_table.csv", index=False)


        GenomeID FeatureType        Location       locus_tag  gene  \
0  NZ_CP045110.1      source  [0:3981941](+)              nd    nd   
1  NZ_CP045110.1         CDS   [348:1563](-)  DGLPADFO_00001  tyrS   
2  NZ_CP045110.1         CDS  [1642:2770](+)  DGLPADFO_00002  anmK   
3  NZ_CP045110.1         CDS  [2897:3743](+)  DGLPADFO_00003  ephA   
4  NZ_CP045110.1         CDS  [3798:4134](-)  DGLPADFO_00004  erpA   

                                      product protein_id isolate host country  \
0                                          nd         nd      nd   nd      nd   
1                       Tyrosine--tRNA ligase         nd      nd   nd      nd   
2         Anhydro-N-acetylmuramic acid kinase         nd      nd   nd      nd   
3                         Epoxide hydrolase A         nd      nd   nd      nd   
4  Iron-sulfur cluster insertion protein ErpA         nd      nd   nd      nd   

  segment      db_xref  
0      nd           nd  
1      nd  COG:COG0162  
2      nd  COG:CO

In [28]:
unzipped_DDBJ = "acinetobacter.fna"

In [29]:
from Bio import SeqIO
import pandas as pd

In [31]:
# Loop through each file and parse
for record in SeqIO.parse(unzipped_DDBJ, "fasta"):
    print(record.id, len(record.seq))

NZ_CP045110.1 3981941
NZ_CP045108.1 7655
NZ_CP045109.1 9540


In [32]:
def infer_sequence_type(header, length):
    h = header.lower()
    if "chromosome" in h or "chr" in h:
        return "Chromosome"
    if "plasmid" in h:
        return "Plasmid"
    if "scaffold" in h:
        return "Scaffold"
    if "contig" in h:
        return "Contig"
    
    # length-based fallback logic
    if length > 1_000_000:
        return "Chromosome"
    elif length > 50_000:
        return "Contig"
    else:
        return "Small_Contig"

def infer_topology(header):
    h = header.lower()
    if "circular" in h:
        return "Circular"
    if "linear" in h:
        return "Linear"
    return "Unknown"

def infer_plasmid_or_chromosome(header):
    h = header.lower()
    if "plasmid" in h:
        return "Plasmid"
    if "chromosome" in h or "chr" in h:
        return "Chromosome"
    return "Unknown"

In [34]:
seq_ids = []
seq_lengths = []
file_names = []
gc_contents = []
at_contents = []
n_counts = []
gc_skews = []
at_skews = []
sequence_types = []
topologies = []
plasmid_chromosome = []
for record in SeqIO.parse(unzipped_DDBJ, "fasta"):
        seq = str(record.seq).upper()
        seq_ids.append(record.id)
        seq_lengths.append(len(record.seq))
        gc = seq.count("G") + seq.count("C")
        at = seq.count("A") + seq.count("T")
        gc_contents.append((seq.count("G") + seq.count("C")) / len(seq) * 100 if len(seq) > 0 else 0)
        at_contents.append((seq.count("A") + seq.count("T")) / len(seq) * 100 if len(seq) > 0 else 0)
        gc_skews.append((seq.count("G") - seq.count("C")) / (gc) * 100 if gc > 0 else 0)
        at_skews.append((seq.count("A") - seq.count("T")) / (at) * 100 if at > 0 else 0)
        header = record.description
        sequence_types.append(infer_sequence_type(header, len(seq)))
        topologies.append(infer_topology(header))
        plasmid_chromosome.append(infer_plasmid_or_chromosome(header))
        n_counts.append(seq.count("N"))
        file_names.append(unzipped_DDBJ)

In [35]:
# Create DataFrame
df = pd.DataFrame({
    "File": file_names,
    "Sequence ID": seq_ids,
    "Sequence Length": seq_lengths,
    "GC_CONTENT" : gc_contents,
    "AT_CONTENT" : at_contents,
    "Ambigious_Base" : n_counts,
    "GC_SKEWS" : gc_skews ,
    "AT_SKEWS" : at_skews ,
    "Sequence_type" : sequence_types,
    "Topology" : topologies, 
    "Chromosome/Plasmid" : plasmid_chromosome
    
})

In [36]:
# Save to CSV
df.to_csv("Acinetobacter_features_1table.csv", index=False)

In [ ]:
#!dir